In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 13

In [4]:
name = list[k].getName()
name

'RP57'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.5f, %.5f] " % ( resultsMC[4], resultsMC[5])


b = "[%.5f, %.5f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.5f, %.5f] " % ( resultSubset[4], resultSubset[5])


In [8]:
mc1 = np.format_float_scientific(np.float32(resultsMC[4]))
mc2 = np.format_float_scientific(np.float32(resultsMC[5]))

ic1 = "["+mc1+", "+mc2+"]"
ic1

'[2.1102887e-02, 3.137837e-02]'

In [9]:
formis1 = np.format_float_scientific(np.float32(resultFORMIS[4]))
formis2 = np.format_float_scientific(np.float32(resultFORMIS[5]))

ic2 = "["+formis1+", "+formis2+"]"
ic2

'[1.1462252e-02, 1.7040242e-02]'

In [10]:
subset1 = np.format_float_scientific(np.float32(resultSubset[4]))
subset2 = np.format_float_scientific(np.float32(resultSubset[5]))

ic3 = "["+subset1+", "+subset2+"]"

ic3

'[2.794609e-02, 3.153391e-02]'

In [11]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.02110, 0.03138] ', None],
 [None, None, None, '[0.02110, 0.03138] ', None],
 [None, None, None, '[0.02110, 0.03138] ', None],
 [None, None, None, '[0.02110, 0.03138] ', None],
 [None, None, None, '[0.02110, 0.03138] ', None],
 [None, None, None, '[0.02110, 0.03138] ', None]]

In [12]:
l = np.array([[1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a],[1, 1, 1, ic3, 1, a]])

In [13]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité de référence de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  "%.2f " % (result[i][2])


results[0][3] = ic1
results[1][3] = "-----------------"
results[2][3] = "-----------------"
results[3][3] = ic2
results[4][3] = ic3


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2]->[var g1 := -x1^2 + x2^3 + 3;var g2 := 2 - x1 - 8 * x2;var g3 := (x1 + 3)^2 + (x2 + 3)^2 - 4;gsys := min(max(g1, g2), g3) ]


,Probabilité de référence de RP57,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,2.8e-02,2.6240628e-02,1.7593716e-03,"[2.1102887e-02, 3.137837e-02",5602,1.20
FORM,2.8e-02,4.168773e-02,1.368773e-02,-----------------,60,0.31
SORM,2.8e-02,3.9656423e-02,1.1656425e-02,-----------------,73,0.38
FORM-IS,2.8e-02,1.4251246e-02,1.3748754e-02,"[1.1462252e-02, 1.7040242e-0",727,0.31
SUBSET,2.8e-02,2.974e-02,1.74e-03,"[2.794609e-02, 3.153391e-02]",20000,1.21


In [14]:
df.to_csv('problem57.csv')